In [1]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import scipy.sparse as sps
%matplotlib inline
%load_ext Cython

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch
import optuna

C:\Users\feder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data_train_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv"
# data_target_user_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv"
data_train_path="data_train.csv"
data_target_user_path="data_target_users_test.csv"
data_train = pd.read_csv(data_train_path)
data_target = pd.read_csv(data_target_user_path)

In [3]:
URM_all =  data_train.pivot(index='row', columns='col', values='data').fillna(0)
item_map = {i : item for i, item in enumerate(URM_all.columns)}
user_map = {i : user for i, user in enumerate(data_target["user_id"])}
item_map_inv = {item : i for i, item in item_map.items()}
user_map_inv = {user : i for i, user in user_map.items()}
missing_index = [x for x in range(1,13025) if x not in URM_all.index.tolist()]
add_urm = pd.DataFrame(index = missing_index, columns = URM_all.columns).fillna(0)
URM_all = pd.concat([URM_all, add_urm]).sort_index()
del add_urm
del missing_index
#data_target["user_id"] = data_target["user_id"]
URM_all = URM_all.to_numpy()
URM_all = sps.csr_matrix(URM_all)
URM_all

<13024x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [4]:
test_folds = 3

URM_trains = [None] * test_folds
evaluator_tests = [None] * test_folds
for i in range(0, test_folds):
    URM_trains[i], URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
    evaluator_tests[i] = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2551 (19.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2581 (19.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2540 (19.5%) Users that have less than 1 test interactions


In [5]:
def train_evaluate(optuna_trial):
    factors = optuna_trial.suggest_int("factors", 1, 1000)
    learning_rate = optuna_trial.suggest_float("learning_rate", 0.0, 0.1, log=True)
    
    mAP = 0.0
    for i in range(0, test_folds):
        SLIM_recommender = MF_MSE_PyTorch(URM_trains[i])
        SLIM_recommender.fit(num_factors=factors, learning_rate=learning_rate, use_cuda=False)
        result_df, _ = evaluator_tests[i].evaluateRecommender(SLIM_recommender)
        mAP += result_df["MAP"].values[0]
    mAP /= test_folds
    return mAP

In [6]:
study = optuna.create_study(direction="maximize")
study.optimize(train_evaluate, n_trials=1)

[I 2023-12-11 22:07:11,483] A new study created in memory with name: no-name-1ec01389-e217-4ea1-85b7-b8c01eb8190d


MF_MSE_PyTorch_Recommender: URM Detected 619 ( 4.8%) users with no interactions.
MF_MSE_PyTorch_Recommender: URM Detected 114 ( 0.5%) items with no interactions.
MF_MSE_PyTorch_Recommender: MF_MSE_PyTorch: Using CPU


C:\Users\feder\AppData\Local\Temp\ipykernel_17708\2649365704.py:3: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  learning_rate = optuna_trial.suggest_uniform("learning_rate", 0.000001, 0.1)
C:\Users\feder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


MF_MSE_PyTorch_Recommender: Epoch 1, Batch: [2993/2993], Samples per second 1775.36
MF_MSE_PyTorch_Recommender: Epoch 1 of 30. Elapsed time 3.61 min


[W 2023-12-11 22:11:11,828] Trial 0 failed with parameters: {'factors': 542, 'learning_rate': 0.09469635707126918} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\feder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\feder\AppData\Local\Temp\ipykernel_17708\2649365704.py", line 9, in train_evaluate
    SLIM_recommender.fit(num_factors=factors, learning_rate=learning_rate, use_cuda=False)
  File "c:\Users\feder\Documents\DEV\my_RecSyS_Course_AT_PoliMi\Recommenders\MatrixFactorization\PyTorch\MF_MSE_PyTorch.py", line 83, in fit
    self._train_with_early_stopping(epochs,
  File "c:\Users\feder\Documents\DEV\my_RecSyS_Course_AT_PoliMi\Recommenders\Incremental_Training_Early_Stopping.py", line 199, in _train_with_early_stopp

KeyboardInterrupt: 